<a href="https://colab.research.google.com/github/NUROISEA/anime-webui-colab/blob/main/notebooks/momocha_mix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2>🌟 <code>Momocha Mix Colab</code></h2>

- 🤔 [How to use](https://github.com/NUROISEA/anime-webui-colab/wiki/Getting-started)
- 🤗 [HuggingFace Repo](https://huggingface.co/NoCrypt/momocha-mix)

In [ ]:
#@title 🚀 `Launch web UI`

model = "momocha-6E7-0.315-AD-E6-0.15-WS.ckpt" # ["momocha-6E7-0.315-AD-E6-0.15-WS.ckpt","momocha-6E7-0.315-AD-E7-0.15-WS.ckpt"]

#@markdown Click the play button on the left to install the web UI and it's dependencies

#@markdown It takes around 3-5 minutes before you can see a `gradio.live` link to the web UI

#@markdown ----
#@markdown Use ngrok?
use_ngrok = False #@param { type:'boolean' }
#@markdown Acquire a ngrok token by following [this guide](https://github.com/NUROISEA/anime-webui-colab/wiki/Using-ngrok).
ngrok_token = "" #@param { type:'string' }
ngrok_region = "auto" #@param ["auto", "ap", "au", "eu", "in", "jp", "sa", "us"]
#@markdown If using ngrok, you will now see a `ngrok.io` link to the web UI

huggingface_repo = "https://huggingface.co/NoCrypt/momocha-mix"

web_ui_folder = "/content/stable-diffusion-webui"
extensions_folder = f"{web_ui_folder}/extensions"
models_folder = f"{web_ui_folder}/models/Stable-diffusion"
vae_folder = f"{web_ui_folder}/models/VAE"
embeddings_folder = f"{web_ui_folder}/embeddings"

ui_config_dictionary = {
  "txt2img/Prompt/value": "masterpiece, best quality, ",
  "txt2img/Negative prompt/value": "lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry"
}

git_flags = "-q"
wget_flags = "-q --show-progress"

vae_link = "https://huggingface.co/NUROISEA/av3-mirror/resolve/main/anything.vae.pt"
vae_name = vae_link.split("/")[-1]

try:
  utility
except NameError:
  !wget -q https://github.com/NUROISEA/anime-webui-colab/raw/main/configs/utility.py -P /content/
  import utility

if not utility.has_run: 
  !pip install -q {utility.xformers_link}
  
  print(f'Cloning stable-diffusion-webui...')
  !git clone {git_flags} -b {utility.webui_branch} https://github.com/NUROISEA/stable-diffusion-webui

  for extension_link in utility.default_extensions:
    extension_name = extension_link.split('/')[-1]
    print(f'Cloning {extension_name}...')
    !git clone {git_flags} {extension_link} {extensions_folder}/{extension_name}
  # !git clone {git_flags} github.com/username/<repo> {extensions_folder}/<repo>

  print(f'Fetching configs...')
  for config_file in utility.default_configs:
    !wget -q {config_file} -P {web_ui_folder}/

  for embedding in utility.default_embeddings:
    !wget {wget_flags} {embedding} -P {embeddings_folder}/
  # !wget {wget_flags} site.com/embedding.pt -P {embeddings_folder}/

  !wget {wget_flags} {vae_link} -P {vae_folder}/
  
  utility.dictionary_to_json(f"{web_ui_folder}/ui-config.json", ui_config_dictionary)
  # utility.dictionary_to_json(f"{web_ui_folder}/config.json", config_dictionary)
  
  !sed -i -e '''/    prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/self.logvar\\[t\\]/self.logvar\\[t.item()\\]/g\"'' /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/models/diffusion/ddpm.py""")''' /content/stable-diffusion-webui/launch.py
  !sed -i -e '''/    prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /content/stable-diffusion-webui/launch.py
   
  utility.has_run = True

if model not in utility.models_downloaded:
  utility.models_downloaded.append(model)
  !wget {wget_flags} {huggingface_repo}/resolve/main/{model} -P {models_folder}/

%cd {web_ui_folder}

#branch = utility.webui_branch
#!git checkout {branch}

arg_list = [
  "--xformers --lowram",
  f"--vae-path {vae_folder}/{vae_name}",
  f"--ckpt {models_folder}/{model}",
]

if use_ngrok:
  arg_list.append(f"--ngrok {ngrok_token}")
  if ngrok_region != "auto":
    arg_list.append(f"--ngrok-region {ngrok_region}")
else:
  arg_list.append("--share")

args = " ".join(arg_list)
!python launch.py {args}

In [ ]:
#@title 💾 `Saving images`

#@markdown The zip file will be visible at the files tab.

#@markdown ----

from datetime import datetime
str_date = datetime.today().strftime('%Y-%m-%d-%H%M%S')
archive_name = f"outputs-{str_date}.zip"

%cd /content/stable-diffusion-webui
print("Zipping...")
!zip -qr /content/{archive_name} outputs
print(f"\033[92mZipped. You can now find {archive_name} at the files tab.\033[0m")

# ----

#@markdown 📲 Copy zip to Google Drive? (Recommended)
copy_to_gdrive = False #@param {type:'boolean'}
#@markdown 📂 The folder path to copy the zip file, if the above is checked, if left blank it will be in `AI/Generated`
gdrive_folder = "AI/Generated" #@param { 'type': 'string' }

if copy_to_gdrive:
  from google.colab import drive

  print("Mounting to Google Drive...")
  drive.mount('/content/drive')
  if gdrive_folder == "":
    gdrive_folder = "AI/Generated"
  
  drive_folder = f"/content/drive/MyDrive/{gdrive_folder}"
  
  !mkdir -p {drive_folder}
  !cp /content/{archive_name} {drive_folder}
  print(f"\033[92mCopied to {gdrive_folder}!\033[0m")

#@markdown ----

#@markdown 📤 Upload zip to a temporary file hosting? 
upload_to_temp = False #@param {type:'boolean'}

#@markdown 🌐 The website/host to upload to. 

#@markdown You can read more about the individual site's limits [here](https://github.com/NUROISEA/anime-webui-colab/wiki/Uploading-to-temporary-file-hosts). 
temp_file_host = "transfer.sh" #@param ["bashupload.com", "keep.sh", "temp.sh", "transfer.sh"]

host_dictionary = {
  "bashupload.com": "https://bashupload.com",
  "keep.sh": "https://free.keep.sh",
  "temp.sh": "https://temp.sh",
  "transfer.sh": f"https://transfer.sh/{archive_name}",
}
host_link = host_dictionary[temp_file_host] # not sure if colab can do this directly at {}

if upload_to_temp:
  print(f"Uploading to {temp_file_host}...")
  !curl -T /content/{archive_name} {host_link}  
  print("\n\033[92m^^^ This is your download link! ^^^\033[0m")

#@markdown ----

#@markdown ❌ Delete the files in the colab? (this wont delete the saved zip)
delete_local_files = False #@param {type:'boolean'}
if delete_local_files:
  %cd /content/stable-diffusion-webui
  !rm -rf outputs
  print("\033[91mDeleted the outputs folder.\033[0m")